In [2]:
import numpy as np
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state
from sklearn.utils.extmath import safe_sparse_dot, squared_norm
from scipy.misc import comb, logsumexp 
from sklearn.linear_model.logistic import _multinomial_grad_hess
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [3]:
mnist = fetch_mldata('MNIST original')
X = mnist.data.astype('float64')
y = mnist.target

In [4]:
train_samples = 30000

In [5]:
random_state = check_random_state(0)

permutation = random_state.permutation(X.shape[0])

X = X[permutation]
y = y[permutation]
X = X.reshape((X.shape[0], -1))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_samples, test_size=10000, random_state=1)

In [7]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [8]:
Y_train = np.zeros((len(y_train), 10))
for i,j in enumerate(y_train):
    Y_train[i, int(j)] = 1

In [9]:
Y_test = np.zeros((len(y_test), 10))
for i,j in enumerate(y_test):
    Y_test[i, int(j)] = 1

Loss function

In [10]:
def loss_function(X,Y, w, alpha=0):
    n_classes = Y.shape[1]
    n_features = X.shape[1]
    m= X.shape[0]
    w = w.reshape(n_classes, -1)
    fit_intercept = w.size == (n_classes * (n_features + 1))
    old_w = w.copy()
    if fit_intercept:
        intercept = w[:, -1]
        w = w[:, :-1]
    else:
        intercept = 0
    p = safe_sparse_dot(X, w.T)
    p += intercept
    p -= logsumexp(p, axis=1)[:, np.newaxis]
    loss = (-1/m)*(Y * p).sum()
    loss += 0.5 * alpha * squared_norm(w)
    p = np.exp(p, p)
    return loss, p, w

In [11]:
def grad_loss(X, Y, w, alpha=0):
    """
    """
    #using sigmoid function from the first hw
    scores = np.dot(X,w)
    scores -= np.max(scores)
    prob = (np.exp(scores).T / np.sum(np.exp(scores),axis=1)).T
    
    grad = -np.dot(X.T,(Y - prob)) + alpha*w
#     print(Y.shape)
#     print(prob.shape)
    return grad

In [12]:
def hessian_loss(X, Y, w, alpha=0):
    """
    """
    #http://www.cedar.buffalo.edu/~srihari/CSE574/Chap4/4.3.4-MultiLogistic.pdf
    #page 19 hessian function of multi logistic regression
    scores = np.dot(X,w)
    scores -= np.max(scores)
    prob = (np.exp(scores).T / np.sum(np.exp(scores),axis=1)).T
    horizontal = []
    vertical = []
    for i in range(0,10):
        for j in range(0,10):
            if i == j:
                ind = 1
            else:
                ind = 0
            
            part = -safe_sparse_dot((X*prob[:,i:(i+1)]).T,X)*ind+safe_sparse_dot((X*prob[:,i:(i+1)]).T,X*prob[:,j:(j+1)])+alpha
            horizontal.append(part)
        
        vertical.append(np.concatenate(horizontal,axis=1))
        horizontal = []
    return np.concatenate(vertical,axis=0)
    

Stochastic Gradient Descent

In [13]:
fit_intercept = True

In [14]:
def sgd(X,Y,w,momentum=0.9, lr=0.01, batch_size=1001, alpha=0.1):
    """
    Real-time forward-mode algorithm using stochastic gradient descent with constant learning 
    rate. Observe that you should only find the optimal learning rate (lr), and 
    penalty parameter (alpha). 
    
    We use the SGD with momentum, which is defined here: 
    http://ufldl.stanford.edu/tutorial/supervised/OptimizationStochasticGradientDescent/
    """
    #first use gd to train hyper parameters
    iterations = 100
    for i in range(0,iterations):
        hyper = hyper_train(X,Y,momentum,lr,batch_size,alpha)
        lr = lr - hyper[0]
        alpha = alpha - hyper[1]
    #use optimized hyper parameters to do sgd
    #initialize a new w
    w = np.random.rand(X_train.shape[1],Y_train.shape[1])
    for i in range(0,iterations):
        #random sample batch
        print("sgs iterations: {}".format(i))
        #random sample. Code copied from https://stackoverflow.com/questions/14262654/numpy-get-random-set-of-rows-from-2d-array
        idx = np.random.randint(X_train.shape[0], size=batch_size)
        X = X_train[idx,:]
        Y = Y_train[idx,:]
        v = momentum*v + lr*grad_loss(X,Y,w,alpha)
        w = w - v
    return w

In [34]:
def hyper_train(X,Y,momentum=0.9, lr=0.01, batch_size=1001, alpha=0.1):
    #initilize w and v
    w = np.zeros((X_train.shape[1],Y_train.shape[1]))
    v = np.zeros((X_train.shape[1],Y_train.shape[1]))
    #random sample. Code copied from https://stackoverflow.com/questions/14262654/numpy-get-random-set-of-rows-from-2d-array
    idx = np.random.randint(X_train.shape[0], size=batch_size)
    X = X_train[idx,:]
    Y = Y_train[idx,:]
    z = np.zeros((X_train.shape[1]*10*2,2))
    iterations = 100
    for i in range(0,iterations):
        
        print("hyper_train iterations: {}".format(i))
        
        #-calculate A
        hessian = hessian_loss(X,Y,w,alpha)
        t1 = np.concatenate([momentum*np.identity(X_train.shape[1]*10),-momentum*np.identity(X_train.shape[1]*10)])
        t2 = np.concatenate([hessian,np.identity(X_train.shape[1]*10)-hessian])
        A = np.concatenate([t1,t2],axis=1)
        #-calculate B
        gradient_nonF = grad_loss(X,Y,w,alpha)
        gradient = gradient_nonF.flatten()
        print(gradient)
        w_F = w.flatten()
        t1 = np.concatenate([gradient,-gradient])
        t2 = np.concatenate([w_F,-w_F])
        B = np.array([t1,t2]).T
        #-calculate Z
        z = np.dot(A,z)+B
        #-Although paper says updating S_t first, A_t and B_t is derivative w.r.t parameters in t-1. Therefore, I calculate A_t,B_t before updating the w andv.
        v = momentum*v + lr*gradient_nonF
        w = w - v
        #-Validation error does not contain v. Gradient of validation error w.r.t S_t is zeroes followed by gradients w.r.t to w.
    grad_val = np.concatenate([np.array([0]*X_train.shape[1]*10),gradient]).reshape(1,X_train.shape[1]*10*2)
    return safe_sparse_dot(grad_val,z)
